In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [2]:
from pyspark.sql import functions as f
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_FullGrouped')\
        .getOrCreate()

22/08/30 10:19:09 WARN Utils: Your hostname, MacBook-Air-de-Willians.local resolves to a loopback address: 127.0.0.1; using 192.168.18.9 instead (on interface en0)
22/08/30 10:19:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/30 10:19:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/30 10:19:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
dataGrouped = spark.read.csv(
    'archive/full_grouped.csv',
    sep = ',',
    header = True,
    )

dataGrouped.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Deaths: string (nullable = true)
 |-- Recovered: string (nullable = true)
 |-- Active: string (nullable = true)
 |-- New cases: string (nullable = true)
 |-- New deaths: string (nullable = true)
 |-- New recovered: string (nullable = true)
 |-- WHO Region: string (nullable = true)



In [5]:
dataGrouped.show(6)

+----------+-------------------+---------+------+---------+------+---------+----------+-------------+--------------------+
|      Date|     Country/Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|          WHO Region|
+----------+-------------------+---------+------+---------+------+---------+----------+-------------+--------------------+
|2020-01-22|        Afghanistan|        0|     0|        0|     0|        0|         0|            0|Eastern Mediterra...|
|2020-01-22|            Albania|        0|     0|        0|     0|        0|         0|            0|              Europe|
|2020-01-22|            Algeria|        0|     0|        0|     0|        0|         0|            0|              Africa|
|2020-01-22|            Andorra|        0|     0|        0|     0|        0|         0|            0|              Europe|
|2020-01-22|             Angola|        0|     0|        0|     0|        0|         0|            0|              Africa|
|2020-01-22|Anti

In [6]:
from pyspark.sql.types import *

data_schema = [
               StructField('Date', DateType(), True),
               StructField('Country/Region', IntegerType(), True),
               StructField('Confirmed', IntegerType(), True),
               StructField('Deaths', IntegerType(), True),
               StructField('Recovered', IntegerType(), True),
               StructField('Active', IntegerType(), True),
               StructField('New cases', IntegerType(), True),
               StructField('New deaths', IntegerType(), True),
               StructField('New recovered', IntegerType(), True),
               StructField('WHO Region', StringType(), True),
            ]

final_struc = StructType(fields=data_schema)

In [7]:
dataFgrouped = spark.read.csv(
    'archive/full_grouped.csv',
    sep = ',',
    header = True,
    schema = final_struc
    )

In [8]:
dataFgrouped.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Country/Region: integer (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- New cases: integer (nullable = true)
 |-- New deaths: integer (nullable = true)
 |-- New recovered: integer (nullable = true)
 |-- WHO Region: string (nullable = true)



In [9]:
dataFgrouped.show(6)

+----------+--------------+---------+------+---------+------+---------+----------+-------------+--------------------+
|      Date|Country/Region|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|          WHO Region|
+----------+--------------+---------+------+---------+------+---------+----------+-------------+--------------------+
|2020-01-22|          null|        0|     0|        0|     0|        0|         0|            0|Eastern Mediterra...|
|2020-01-22|          null|        0|     0|        0|     0|        0|         0|            0|              Europe|
|2020-01-22|          null|        0|     0|        0|     0|        0|         0|            0|              Africa|
|2020-01-22|          null|        0|     0|        0|     0|        0|         0|            0|              Europe|
|2020-01-22|          null|        0|     0|        0|     0|        0|         0|            0|              Africa|
|2020-01-22|          null|        0|     0|        0|  

### Generando file parquet

In [11]:
dataFgrouped.write.save('data/parquets/fullGrouped', format='parquet')

In [ ]:
dataFgrouped.describe().toPandas()

### Agrupando datos por WHO Region

In [ ]:
dataFgrouped.select(['WHO Region', 'Confirmed', 'Deaths', 'Recovered', 'Active']).groupBy('WHO Region').mean().show()

### Visualizando datos

In [ ]:
sec_df =  dataFgrouped.select(['WHO Region', 'Confirmed', 'Deaths', 'Recovered', 'Active']).groupBy('WHO Region').mean().toPandas()

In [ ]:
sec_df

In [ ]:
sec_df.plot(kind = 'bar', x='WHO Region', y = sec_df.columns.tolist()[1:], figsize=(12, 6))

In [ ]:
from pyspark.sql.functions import col, lit

In [ ]:
dataFilter = dataFgrouped.filter( (col('WHO Region') == 'Europe') )

In [ ]:
dataFilter.show()

In [ ]:
sec_df1 =  dataFilter.select(['Country/Region', 'Deaths', 'Active', 'New cases']).groupBy('Country/Region').mean().toPandas()

In [ ]:
sec_df1